<a href="https://colab.research.google.com/github/tsa87/semi-jtvae/blob/master/semi_jtvae_train_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q torch==1.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install -q torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q dive-into-graphs

In [2]:
!pip install -q p_tqdm
!pip install -q jupyter
!pip install -q ipywidgets widgetsnbextension pandas-profiling

In [3]:
%load_ext autoreload
%autoreload 2
import os
import json
import rdkit
import torch
import pickle 
import argparse
import numpy as np

from dig.ggraph.dataset import ZINC250k, ZINC800
from molecule_optimizer.externals.fast_jtnn.datautils import SemiMolTreeFolder
from molecule_optimizer.runner.semi_jtvae import SemiJTVAEGeneratorPredictor
from torch_geometric.data import DenseDataLoader

import warnings
warnings.filterwarnings("ignore")

In [4]:
lg = rdkit.RDLogger.logger() 
lg.setLevel(rdkit.RDLogger.CRITICAL)

In [5]:
conf = json.load(open("training/configs/rand_gen_zinc250k_config_dict.json"))

In [6]:
if False:
    print("Processing Dataset...")
    _ = ZINC250k(
        root=conf["data"]["root"],
        one_shot=False,
        use_aug=False,
    )

In [7]:
if False:
    zinc_250_jt = torch.load(
        os.path.join(conf["data"]["root"], conf["data"]["processed_path"])
    )
    smiles = zinc_250_jt[-1]
    labels = zinc_250_jt[0].y

In [8]:
if False:
    runner = SemiJTVAEGeneratorPredictor(smiles, labels)
    runner.get_model(
        "rand_gen",
        {
            "hidden_size": conf["model"]["hidden_size"],
            "latent_size": conf["model"]["latent_size"],
            "depthT": conf["model"]["depthT"],
            "depthG": conf["model"]["depthG"],
            "label_size": 1,
            "label_mean": float(torch.mean(labels)),
            "label_var": float(torch.var(labels)),
        },
    )
    with open('runner.pickle', 'wb') as f: 
        pickle.dump(runner, f)

In [9]:
with open('runner.pickle', 'rb') as f: 
    runner = pickle.load(f)

In [11]:
loader = SemiMolTreeFolder(
      runner.smiles,
      runner.labels,
      runner.vocab,
      conf["batch_size"],
      num_workers=conf["num_workers"],
  )

In [ ]:
print("Training model...")
runner.train_gen_pred(
    loader=loader,
    load_epoch=0,
    lr=conf["lr"],
    anneal_rate=conf["anneal_rate"],
    clip_norm=conf["clip_norm"],
    num_epochs=conf["num_epochs"],
    alpha=conf["alpha"],
    beta=conf["beta"],
    max_beta=conf["max_beta"],
    step_beta=conf["step_beta"],
    anneal_iter=conf["anneal_iter"],
    kl_anneal_iter=conf["kl_anneal_iter"],
    print_iter=conf["print_iter"],
    save_iter=conf["save_iter"],
)